In [1]:
import os
from ortools.constraint_solver import pywrapcp, routing_enums_pb2
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def parse_instance_file(file_name):
    coordinates = {}
    demands = {}
    time_windows = {}
    stand_times = {}
    dimension = None
    capacity = None
    capacity_vol = None

    with open(file_name, 'r') as file:
        lines = file.readlines()

    # Skip the initial comment section marked by ###
    lines = iter(lines)

    section = None
    for line in lines:
        line = line.strip()
        if line.startswith("DIMENSION"):
            dimension = int(line.split(':')[1].strip())
        elif line.startswith("CAPACITY "):
            capacity = int(line.split(':')[1].strip())
        elif line.startswith("CAPACITY_VOL"):
            capacity_vol = int(line.split(':')[1].strip())
        elif line == "NODE_COORD_SECTION":
            section = "coordinates"
            continue
        elif line == "PICKUP_SECTION":
            section = "pickups"
            continue            
        elif line == "DEMAND_SECTION":
            section = "demands"
            continue
        elif line == "TIME_WINDOW_SECTION":
            section = "time_windows"
            continue
        elif line == "STANDTIME_SECTION":
            section = "stand_times"
            continue
        elif line == "DEPOT_SECTION":
            section = "depot"
            continue
        elif line == "EOF":
            break

        if section == "coordinates" and line:
            parts = line.split()
            node_id, x, y = int(parts[0]) - 1, int(parts[1]), int(parts[2])
            coordinates[node_id] = (x, y)
        elif section == "demands" and line:
            parts = line.split()
            node_id, demand = int(parts[0]) - 1, int(parts[1])
            demands[node_id] = demand
        elif section == "time_windows" and line:
            parts = line.split()
            node_id = int(parts[0]) - 1
            begin_hh, begin_mm = map(int, parts[1].split(':'))
            end_hh, end_mm = map(int, parts[2].split(':'))
            time_windows[node_id] = (begin_hh * 60 + begin_mm, end_hh * 60 + end_mm)
        elif section == "stand_times" and line:
            parts = line.split()
            node_id, stand_time = int(parts[0]) - 1, int(parts[1])
            stand_times[node_id] = stand_time
        elif section == "depot" and line.isdigit():
            depot = int(line) - 1
    time_windows[0]=(0, 1440)
    return coordinates, demands, time_windows, stand_times, dimension, capacity, capacity_vol

def create_time_matrix(coordinates):
    locations = list(coordinates.values())
    time_matrix = {}
    speed = 1000  # Speed in km/h
    for from_index, from_node in enumerate(locations):
        time_matrix[from_index] = {}
        for to_index, to_node in enumerate(locations):
            if from_index == to_index:
                time_matrix[from_index][to_index] = 0
            else:
                distance = np.linalg.norm(np.array(from_node) - np.array(to_node))   # Distance in km
                time_matrix[from_index][to_index] = int(distance / speed * 60)  # Time in minutes
    return time_matrix


In [3]:
def print_solution(manager, routing, solution):
    time_dimension = routing.GetDimensionOrDie('Time')
    total_time = 0
    total_load = 0
    active_vehicles = 0
    for vehicle_id in range(routing.vehicles()):
        index = routing.Start(vehicle_id)
        plan_output = f'Route for vehicle {vehicle_id}:'
        route_time = 0
        route_load = 0
        route_nodes = []
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_nodes.append(node_index)  # Adjust node index to match original numbering
            route_load += data['demands'][node_index]
            previous_index = index
            index = solution.Value(routing.NextVar(index))
        route_time = solution.Value(time_dimension.CumulVar(index))
        if route_time > 0:
            active_vehicles += 1
            plan_output += ' -> '.join(map(str, route_nodes))
            plan_output += f'\nTime of the route: {route_time}m\n'
            plan_output += f'Load of the route: {route_load}\n'
            print(plan_output)
        total_time += route_time
        total_load += route_load
    print(f'Total time of all routes: {total_time}m')
    print(f'Total load of all routes: {total_load}')
    print(f'Total vehicles used: {active_vehicles}')

In [4]:
def plot_solution(coordinates, routes, times, time_windows):
    plt.figure(figsize=(30, 30))

    for vehicle_id, route in enumerate(routes):
        if len(route) <= 1:
            continue

        x_coords = [coordinates[node][0] for node in route]
        y_coords = [coordinates[node][1] for node in route]

        plt.plot(x_coords, y_coords, marker='o', label=f'Vehicle {vehicle_id}')

        for i, node in enumerate(route):
            x, y = coordinates[node]
            if i == 0:
                plt.text(x, y, f'{node}\nDepot', fontsize=8, ha='center')
            else:
                arrival_time = times[vehicle_id][i - 1]
                start, end = time_windows[node]
                plt.text(x, y, f'{node}\n{arrival_time // 60:02}:{arrival_time % 60}\n[{start // 60:02}:{start % 60}-{end // 60:02}:{end % 60}]',
                         fontsize=8, ha='center')

    plt.legend()
    plt.xlabel('X Coordinate')
    plt.ylabel('Y Coordinate')
    plt.title('Vehicle Routes')
    plt.grid()
    plt.savefig('routes.png')
    plt.show()

In [5]:
def extract_solution(manager, routing, solution, time_dimension):
    routes = []
    times = []
    for vehicle_id in range(routing.vehicles()):
        index = routing.Start(vehicle_id)
        route = []
        time = []
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route.append(node_index)
            time.append(solution.Value(time_dimension.CumulVar(index)))
            index = solution.Value(routing.NextVar(index))
        routes.append(route)
        times.append(time)
    return routes, times

In [6]:
#os.chdir('Day24')  # Change to the appropriate folder
file_name = 'instance_clean.txt'

# Parse input data
coordinates, demands, time_windows, stand_times, dimension, capacity, capacity_vol = parse_instance_file(file_name)

# Define data model
data = {}
data['time_matrix'] = create_time_matrix(coordinates)
data['demands'] = list(demands.values())
data['time_windows'] = list(time_windows.values())
data['stand_times'] = list(stand_times.values())
# Example: 10 vehicles with given capacity
data['vehicle_capacities'] = [capacity] * 50
data['num_vehicles'] = len(data['vehicle_capacities'])
data['depot'] = 0


In [7]:
# Create the routing index manager
manager = pywrapcp.RoutingIndexManager(len(data['time_matrix']), data['num_vehicles'], data['depot'])

# Create Routing Model
#routing = pywrapcp.RoutingModel(manager)

routing_parameters = pywrapcp.DefaultRoutingModelParameters()
routing_parameters.solver_parameters.trace_propagation = True
routing_parameters.solver_parameters.trace_search = True
routing = pywrapcp.RoutingModel(manager)#, routing_parameters

# Create and register a transit callback
def time_callback(from_index, to_index):
    # Returns the travel time between the two nodes
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['time_matrix'][from_node][to_node] + data['stand_times'][from_node]

transit_callback_index = routing.RegisterTransitCallback(time_callback)

# Define cost of each arc
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

# Add Capacity constraint
def demand_callback(from_index):
    # Returns the demand of the node
    from_node = manager.IndexToNode(from_index)
    return data['demands'][from_node]

demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
routing.AddDimensionWithVehicleCapacity(
    demand_callback_index,
    0,  # Null capacity slack
    data['vehicle_capacities'],  # Vehicle maximum capacities
    True,  # Start cumul to zero
    'Capacity')

# Add Time Window constraint
routing.AddDimension(
    transit_callback_index,
    1440,  # Allow waiting time
    1440,  # Maximum time per vehicle
    False,  # Don't force start cumul to zero
    'Time')


time_dimension = routing.GetDimensionOrDie('Time')
for location_idx, time_window in enumerate(data['time_windows']):
    if location_idx == data['depot']:
        continue
    index = manager.NodeToIndex(location_idx)
    time_dimension.CumulVar(index).SetRange(time_window[0], time_window[1])
# Add time window constraints for each vehicle start node.
depot_idx = data["depot"]
for vehicle_id in range(data["num_vehicles"]):
    index = routing.Start(vehicle_id)
    time_dimension.CumulVar(index).SetRange(
        data["time_windows"][depot_idx][0], data["time_windows"][depot_idx][1]
    )
for i in range(data["num_vehicles"]):
    routing.AddVariableMinimizedByFinalizer(
        time_dimension.CumulVar(routing.Start(i))
    )
    routing.AddVariableMinimizedByFinalizer(time_dimension.CumulVar(routing.End(i)))


# Setting search heuristic to Tabu Search
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
#search_parameters.local_search_metaheuristic = (
#   routing_enums_pb2.LocalSearchMetaheuristic.TABU_SEARCH)
search_parameters.log_search = True


search_parameters.time_limit.seconds = 300


# Setting search heuristic to Greedy Algorithm
#search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
#search_parameters.time_limit.seconds = 30



In [8]:
def print_solution(manager, routing, solution, time_dimension, demands, time_windows):
    print("Solution Details:\n")
    active_vehicles = 0

    for vehicle_id in range(routing.vehicles()):
        index = routing.Start(vehicle_id)
        route = []
        times = []
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            arrival_time = solution.Value(time_dimension.CumulVar(index))
            route.append(
                (
                    node_index,
                    arrival_time,
                    time_windows[node_index],
                    demands[node_index],
                )
            )
            index = solution.Value(routing.NextVar(index))

        # Only print routes with non-zero length
        if len(route) > 1:
            active_vehicles += 1
            print(f"Vehicle {vehicle_id}:\n")
            for node, arrival, time_window, demand in route:
                #arrival_time_str = f"{arrival // 60:02}:{arrival % 60:02}"
                arrival_time_str = f"{arrival}"
                time_window_str = (
#                    f"[{time_window[0] // 60:02}:{time_window[0] % 60:02} - "
#                    f"{time_window[1] // 60:02}:{time_window[1] % 60:02}]"
                    f"[{time_window[0]} - "
                    f"{time_window[1]}]"
                    
                )
                print(
                    f"\tNode {node}, Arrival: {arrival_time_str}, "
                    f"Time Window: {time_window_str}, Demand: {demand}"
                )
            print("\n")

    print(f"Total active vehicles: {active_vehicles}\n")


In [9]:
# Solve the problem
print("Starting the solver...")
solution = routing.SolveWithParameters(search_parameters)

# Print solution and visualize
if solution:
    print("Solution found! Printing solution...")
    routes, times = extract_solution(manager, routing, solution, time_dimension)
    #plot_solution(coordinates, routes, times, data['time_windows'])
    print_solution(manager, routing, solution, time_dimension, demands, time_windows)
else:
    print("No solution found!")

Starting the solver...
Solution found! Printing solution...
Solution Details:

Vehicle 42:

	Node 0, Arrival: 420, Time Window: [0 - 1440], Demand: 0
	Node 66, Arrival: 510, Time Window: [885 - 915], Demand: 3400


Vehicle 43:

	Node 0, Arrival: 420, Time Window: [0 - 1440], Demand: 0
	Node 94, Arrival: 555, Time Window: [885 - 945], Demand: 2120
	Node 89, Arrival: 555, Time Window: [870 - 900], Demand: 1140
	Node 48, Arrival: 555, Time Window: [840 - 885], Demand: 1390


Vehicle 44:

	Node 0, Arrival: 420, Time Window: [0 - 1440], Demand: 0
	Node 145, Arrival: 465, Time Window: [420 - 1080], Demand: 4910
	Node 98, Arrival: 615, Time Window: [810 - 870], Demand: 4250
	Node 54, Arrival: 615, Time Window: [420 - 1080], Demand: 4970
	Node 52, Arrival: 615, Time Window: [420 - 1080], Demand: 4750
	Node 41, Arrival: 615, Time Window: [750 - 780], Demand: 4510
	Node 37, Arrival: 615, Time Window: [420 - 1080], Demand: 3880
	Node 14, Arrival: 720, Time Window: [420 - 1080], Demand: 1420
	Node

In [10]:
time_matrix = create_time_matrix(coordinates)

In [16]:
time_matrix

{0: {0: 0,
  1: 92,
  2: 91,
  3: 89,
  4: 89,
  5: 89,
  6: 89,
  7: 87,
  8: 89,
  9: 89,
  10: 88,
  11: 88,
  12: 88,
  13: 87,
  14: 87,
  15: 88,
  16: 87,
  17: 87,
  18: 86,
  19: 85,
  20: 86,
  21: 86,
  22: 86,
  23: 84,
  24: 85,
  25: 85,
  26: 85,
  27: 84,
  28: 83,
  29: 84,
  30: 83,
  31: 84,
  32: 83,
  33: 84,
  34: 83,
  35: 83,
  36: 83,
  37: 81,
  38: 83,
  39: 82,
  40: 82,
  41: 82,
  42: 82,
  43: 81,
  44: 82,
  45: 81,
  46: 81,
  47: 81,
  48: 81,
  49: 81,
  50: 81,
  51: 80,
  52: 81,
  53: 81,
  54: 80,
  55: 80,
  56: 80,
  57: 80,
  58: 79,
  59: 79,
  60: 78,
  61: 78,
  62: 76,
  63: 77,
  64: 77,
  65: 77,
  66: 77,
  67: 77,
  68: 77,
  69: 76,
  70: 77,
  71: 76,
  72: 76,
  73: 75,
  74: 73,
  75: 74,
  76: 74,
  77: 74,
  78: 74,
  79: 74,
  80: 74,
  81: 73,
  82: 73,
  83: 73,
  84: 73,
  85: 72,
  86: 73,
  87: 71,
  88: 71,
  89: 71,
  90: 69,
  91: 70,
  92: 70,
  93: 67,
  94: 67,
  95: 67,
  96: 66,
  97: 66,
  98: 66,
  99: 65,
  100: 6